This file is the effort trying to cluster the topics based on LDA model. This is not in use anymore. However, I also think about (maybe) we can apply it in the future.


In [ ]:
#Clean data
import pandas as pd 
import os
data =pd.read_csv('https://raw.githubusercontent.com/npnkhoi/github-data-analysis/main/2k_random_repos.csv')
data.drop(data.columns[[0]], axis=1, inplace=True)
data.columns = ['full_name','num_stars','num_forks', 'num_stars1', 'main_language', 'created_at','updated_at','license_key','description','num_open_issues','topics']
data = data.drop(columns = ['num_stars1'])
data2 = data.drop(columns=['full_name','num_stars','num_forks','created_at','updated_at','license_key','num_open_issues'], axis=1)
data2.dropna()



,main_language,description,topics
0,Kotlin,Android app for streaming and downloading Movi...,"android, anime, media-center, streaming-app, v..."
1,C++,The fastest and most memory efficient lattice ...,"cfd, computational-fluid-dynamics, fluid-simul..."
2,Go,A tool for visualizing differences between two...,"editor-tools, pdf, pdf-diff, pdf-difference, t..."
9,Python,nps认证绕过利用工具，使用此工具可在浏览器访问web控制端后台页面，或者批量获取socks...,"bypass, fofa, nps, proxy"
12,Go,"A tool for showing disk usage. (Linux, MacOS a...","cli, command-line, disk-space, diskusage, gola..."
...,...,...,...
1985,Kotlin,A polyfill to use WebAssembly in React Native.,"android, ios, polyfill, react, react-native, w..."
1986,SCSS,"A compact, better looking, top-left user area.","betterdiscord, betterdiscord-theme, discord, d..."
1987,R,By-hand code for models and algorithms. An upd...,"algorithms, bayesian, cox-model, expectation-m..."
1989,TypeScript,Plugin for Obsidian: Use shortcuts for input s...,"obsidian, obsidian-plugin"


In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data_here2 = data2['topics'].dropna().tolist()
data_words = list(sent_to_words(data_here2))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:100])

from pprint import pprint

# number of topics
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1)]


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"api" + 0.013*"framework" + 0.010*"web" + 0.008*"python" + '
  '0.008*"data" + 0.008*"hacktoberfest" + 0.007*"typescript" + 0.006*"linux" + '
  '0.006*"plus" + 0.006*"windows"'),
 (1,
  '0.012*"telegram" + 0.011*"neural" + 0.011*"bot" + 0.010*"python" + '
  '0.009*"learning" + 0.008*"data" + 0.008*"android" + 0.008*"network" + '
  '0.007*"app" + 0.006*"awesome"'),
 (2,
  '0.020*"hacktoberfest" + 0.009*"java" + 0.009*"windows" + 0.008*"javascript" '
  '+ 0.007*"python" + 0.007*"plugin" + 0.007*"learning" + 0.007*"trading" + '
  '0.006*"css" + 0.006*"react"'),
 (3,
  '0.018*"discord" + 0.011*"learning" + 0.010*"api" + 0.009*"pytorch" + '
  '0.009*"dubbo" + 0.008*"react" + 0.007*"wordpress" + 0.007*"server" + '
  '0.006*"machine" + 0.006*"tool"'),
 (4,
  '0.021*"hacktoberfest" + 0.011*"android" + 0.009*"learning" + 0.007*"php" + '
  '0.007*"angular" + 0.006*"machine" + 0.006*"api" + 0.006*"engine" + '
  '0.006*"sm" + 0.006*"json"'),
 (5,
  '0.012*"android" + 0.011*"python" +

In [ ]:
result = list(lda_model.print_topics())

In [ ]:
import re 
topic_info = []
for line in result:
  result_dict = {}
  info = line[1].split("+")
  for string in info: 
    string=string.strip().replace("/'", "").replace('"', '').split("*")
    key, val = string[1], string[0]
    result_dict[key]=float(val)
  topic_info.append(result_dict)

In [ ]:
for dic in topic_info:
  print(dic)

{'api': 0.013, 'framework': 0.013, 'web': 0.01, 'python': 0.008, 'data': 0.008, 'hacktoberfest': 0.008, 'typescript': 0.007, 'linux': 0.006, 'plus': 0.006, 'windows': 0.006}
{'telegram': 0.012, 'neural': 0.011, 'bot': 0.011, 'python': 0.01, 'learning': 0.009, 'data': 0.008, 'android': 0.008, 'network': 0.008, 'app': 0.007, 'awesome': 0.006}
{'hacktoberfest': 0.02, 'java': 0.009, 'windows': 0.009, 'javascript': 0.008, 'python': 0.007, 'plugin': 0.007, 'learning': 0.007, 'trading': 0.007, 'css': 0.006, 'react': 0.006}
{'discord': 0.018, 'learning': 0.011, 'api': 0.01, 'pytorch': 0.009, 'dubbo': 0.009, 'react': 0.008, 'wordpress': 0.007, 'server': 0.007, 'machine': 0.006, 'tool': 0.006}
{'hacktoberfest': 0.021, 'android': 0.011, 'learning': 0.009, 'php': 0.007, 'angular': 0.007, 'machine': 0.006, 'api': 0.006, 'engine': 0.006, 'sm': 0.006, 'json': 0.006}
{'android': 0.012, 'python': 0.011, 'cost': 0.007, 'music': 0.007, 'compression': 0.007, 'learning': 0.007, 'security': 0.007, 'html': 0

In [ ]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/datalab'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/datalab'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.018762  0.050837       1        1  18.421400
3     -0.045891  0.049902       2        1  17.678880
5      0.103079  0.032012       3        1  16.403524
0     -0.114335 -0.000423       4        1  15.973008
1      0.068752  0.013686       5        1  15.828021
4      0.007158 -0.146014       6        1  15.695167, topic_info=               Term       Freq      Total Category  logprob  loglift
1031       telegram   8.000000   8.000000  Default  30.0000  30.0000
144         discord  22.000000  22.000000  Default  29.0000  29.0000
66    hacktoberfest  45.000000  45.000000  Default  28.0000  28.0000
1466          dubbo   7.000000   7.000000  Default  27.0000  27.0000
350       framework  14.000000  14.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
144         discord   3.328528  22.474196   Topic6  -5.3160  -0.0580
82           neural   2.830189  14.361383   Topic6  -5.4782   0.2276
212      javascript   2.916858  20.267388   Topic6  -5.4480  -0.0867
67              ios   2.681889  13.068817   Topic6  -5.5320   0.2681
204          server   2.682346  14.546322   Topic6  -5.5318   0.1612

[389 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
185       4  0.892956      absinthe
160       1  0.432445      accepted
160       4  0.144148      accepted
160       5  0.144148      accepted
160       6  0.144148      accepted
...     ...       ...           ...
1407      5  0.302383      workflow
1407      6  0.604767      workflow
1832      4  0.491338           wpf
483       3  0.940022           xml
590       6  0.504886  zettelkasten

[613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 6, 1, 2, 5])

In [ ]:
full_dict = ["api" ,"framework","web" ,"python","data" ,"hacktoberfest" ,"typescript","linux","plus" ,"windows","telegram" ,"neural" ,"bot" ,"python","learning","data","android","network","app" ,"awesome","hacktoberfest","java","windows","javascript","python","plugin","learning" ,"trading" ,"css","react","discord","learning","api" ,"pytorch","dubbo" ,"react" ,"wordpress","server" , "machine" ,"tool","hacktoberfest","android","learning","php","angular","machine","api","engine","sm","json","android","python","cost","music","compression","learning","security" ,"html" ,
"cloud" ,"extension"]

In [ ]:
domain_list=[]
#list contains domain for corresponding row
for words in data2['topics']:
  words_stripped = []
  words=str(words)
  if words =='nan':
    continue
  else:
    domain_list.append([word.strip() for word in words.split(',')])
    #for word in words.split(','):
      #word = word.strip()




[['android', 'anime', 'media-center', 'streaming-app', 'video-streaming'], ['cfd', 'computational-fluid-dynamics', 'fluid-simulation', 'fluid-solver', 'gpgpu', 'gpu', 'gpu-acceleration', 'gpu-computing', 'graphics-library', 'high-performance-computing', 'hpc', 'interactive-visualization', 'lattice-boltzmann-method', 'lbm', 'opencl', 'physics', 'raytracing', 'scientific-computing', 'scientific-visualization', 'simulation'], ['editor-tools', 'pdf', 'pdf-diff', 'pdf-difference', 'typesetter'], ['awesome-list', 'awesome-resources', 'data-science', 'internships', 'machine-learning'], ['bypass', 'fofa', 'nps', 'proxy'], ['cli', 'command-line', 'disk-space', 'diskusage', 'golang', 'linux', 'macos', 'tool', 'windows'], ['artificial-intelligence', 'attention-mechanisms', 'deep-learning'], ['hacktoberfest', 'ios', 'swift'], ['bioinformatics', 'sequence-alignment'], ['batch-processing', 'k8s', 'kubernetes', 'spark'], ['course-project', 'program-verification'], ['implicit-neural-representation', '